<a href="https://colab.research.google.com/github/jscienciadados/metodos-ensemble/blob/main/Otimiza_Parametros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Machine Learning



## Otimização dos Parâmetros com Randomized Search¶

Todo modelo de Machine Learning possui parâmetros que permitem a customização do modelo. Esses parâmetros também são chamados de hiperparâmetros.

Em programação os algoritmos de Machine Learning são representados por funções e cada função possui os parâmetros de customização, exatamente o que chamamos de hiperparâmetros.

É comum ainda que as pessoas se refiram aos coeficientes do modelo (encontrados ao final do treinamento) como parâmetros.

Parte do nosso trabalho como Cientistas de Dados é encontrar a melhor combinação de hiperparâmetros para cada modelo.

Em Métodos Ensemble esse trabalho é ainda mais complexo, pois temos os hiperparâmetros do estimador base e os hiperparâmetros do modelo ensemble, conforme este exemplo abaixo:

    Estimador base:

estim_base = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski', metric_params=None, n_jobs=None, n_neighbors=5, p=2, weights='uniform')

    Modelo Ensemble:

BaggingClassifier(base_estimator=estim_base, bootstrap=True, bootstrap_features=False, max_features=0.5, max_samples=0.5, n_estimators=10, n_jobs=None, oob_score=False, random_state=None, verbose=0, warm_start=False)

## Extremely Randomized Forest

Modelo Padrão, com hiperparâmetro escolhidos manualmente.

In [1]:
# Imports
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Buscando dado diretor do Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
credit = pd.read_excel("/content/drive/MyDrive/datasets/credit.xls", skiprows = 1)

In [5]:
print(credit)

          ID  LIMIT_BAL  SEX  ...  PAY_AMT5  PAY_AMT6  default payment next month
0          1      20000    2  ...         0         0                           1
1          2     120000    2  ...         0      2000                           1
2          3      90000    2  ...      1000      5000                           0
3          4      50000    2  ...      1069      1000                           0
4          5      50000    1  ...       689       679                           0
...      ...        ...  ...  ...       ...       ...                         ...
29995  29996     220000    1  ...      5000      1000                           0
29996  29997     150000    1  ...         0         0                           0
29997  29998      30000    1  ...      2000      3100                           1
29998  29999      80000    1  ...     52964      1804                           1
29999  30000      50000    1  ...      1000      1000                           1

[30000 rows x 2

In [7]:
# Variável target
target = 'default payment next month'
y = np.asarray(credit[target])



In [8]:
# Variáveis preditoras
features = credit.columns.drop(['ID', target])
X = np.asarray(credit[features])

In [9]:
# Divisão de dados de treino e de teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 99)



In [10]:
# Classificador
clf = ExtraTreesClassifier(n_estimators = 500, random_state = 99)



# Treinamento do Modelo

In [11]:
clf.fit(X_train, y_train)

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=500,
                     n_jobs=None, oob_score=False, random_state=99, verbose=0,
                     warm_start=False)

In [12]:
# Score
scores = cross_val_score(clf, X_train, y_train, cv = 3, scoring = 'accuracy', n_jobs = -1)



In [13]:
# Imprimindo o resultado
print ("ExtraTreesClassifier -> Acurácia em Treino: Média = %0.3f Desvio Padrão = %0.3f" % (np.mean(scores), np.std(scores)))

ExtraTreesClassifier -> Acurácia em Treino: Média = 0.812 Desvio Padrão = 0.002


In [14]:
# Fazendo previsões
y_pred = clf.predict(X_test)



In [15]:
# Confusion Matrix
confusionMatrix = confusion_matrix(y_test, y_pred)
print (confusionMatrix)



[[6532  446]
 [1273  749]]


In [16]:
# Acurácia
print("Acurácia em Teste:", accuracy_score(y_test, y_pred))



Acurácia em Teste: 0.809


## Otimização dos Hiperparâmetros com Randomized Search

http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html

O Randomized Search gera amostras dos parâmetros dos algoritmos a partir de uma distribuição randômica uniforme para um número fixo de interações. Um modelo é construído e testado para cada combinação de parâmetros.

In [17]:
# Import
from sklearn.model_selection import RandomizedSearchCV

In [18]:
# Definição dos parâmetros
param_dist = {"max_depth": [1, 3, 7, 8, 12, None],
              "max_features": [8, 9, 10, 11, 16, 22],
              "min_samples_split": [8, 10, 11, 14, 16, 19],
              "min_samples_leaf": [1, 2, 3, 4, 5, 6, 7],
              "bootstrap": [True, False]}

# Para o classificador criado com ExtraTrees, testamos diferentes combinações de parâmetros
rsearch = RandomizedSearchCV(clf, param_distributions = param_dist, n_iter = 25, return_train_score = True)  

# Aplicando o resultado ao conjunto de dados de treino e obtendo o score
rsearch.fit(X_train, y_train)

# Resultados
rsearch.cv_results_

# Imprimindo o melhor estimador
bestclf = rsearch.best_estimator_
print (bestclf)

# Aplicando o melhor estimador para realizar as previsões
y_pred = bestclf.predict(X_test)

# Confusion Matrix
confusionMatrix = confusion_matrix(y_test, y_pred)
print(confusionMatrix)

# Acurácia
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)



ExtraTreesClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=7, max_features=16,
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=11,
                     min_weight_fraction_leaf=0.0, n_estimators=500,
                     n_jobs=None, oob_score=False, random_state=99, verbose=0,
                     warm_start=False)
[[6648  330]
 [1289  733]]
0.8201111111111111


In [20]:
# Obtem o grid com todas as combinações de parametros
#rsearch.cv_results_

## Grid Search x Randomized Search para Estimação dos Hiperparâmetros

http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html

In [21]:
import numpy as np
from time import time
from scipy.stats import randint as sp_randint
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import load_digits

# Obtém o dataset
digits = load_digits()
X, y = digits.data, digits.target

# Construindo o classificador
clf = RandomForestClassifier(n_estimators = 20)



In [22]:
# Randomized Search

# Valores dos parâmetros que serão testados
param_dist = {"max_depth": [3, None],
              "max_features": sp_randint(1, 11),
              "min_samples_leaf": sp_randint(1, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# Executando o Randomized Search
n_iter_search = 20
random_search = RandomizedSearchCV(clf,
                                   param_distributions = param_dist,
                                   n_iter = n_iter_search,
                                   return_train_score=True)

start = time()
random_search.fit(X, y)
print("RandomizedSearchCV executou em %.2f segundos para %d candidatos a parâmetros do modelo."
      % ((time() - start), n_iter_search))

# Imprime as combinações dos parâmetros e suas respectivas médias de acurácia
random_search.cv_results_



RandomizedSearchCV executou em 6.71 segundos para 20 candidatos a parâmetros do modelo.


{'mean_fit_time': array([0.05796118, 0.04282651, 0.05970535, 0.10232582, 0.08563981,
        0.07552214, 0.05962601, 0.04763293, 0.0291379 , 0.0335588 ,
        0.03711009, 0.11332502, 0.03856792, 0.07748218, 0.02933559,
        0.03084254, 0.03471351, 0.03660641, 0.03613505, 0.08170238]),
 'mean_score_time': array([0.00347185, 0.00339947, 0.00357633, 0.00366263, 0.00353289,
        0.00363317, 0.00350018, 0.00338612, 0.00318618, 0.00311928,
        0.00293508, 0.00373106, 0.00295625, 0.00353384, 0.00298357,
        0.00290494, 0.00370512, 0.00298715, 0.00340838, 0.00352788]),
 'mean_test_score': array([0.90042092, 0.78798979, 0.92432993, 0.93268493, 0.89873414,
        0.91933612, 0.90877128, 0.79468121, 0.81357629, 0.85085887,
        0.79911483, 0.91878211, 0.81693284, 0.92322346, 0.79411018,
        0.76631383, 0.89482823, 0.80304859, 0.86257041, 0.91043175]),
 'mean_train_score': array([0.99053903, 0.87075763, 0.99819164, 1.        , 0.97760082,
        0.9993044 , 0.99234787, 0.8

In [23]:
# Grid Search

# Usando um grid completo de todos os parâmetros
param_grid = {"max_depth": [3, None],
              "max_features": [1, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# Executando o Grid Search
grid_search = GridSearchCV(clf, param_grid = param_grid, return_train_score = True)
start = time()
grid_search.fit(X, y)

print("GridSearchCV executou em %.2f segundos para todas as combinações de candidatos a parâmetros do modelo."
      % (time() - start))
grid_search.cv_results_



GridSearchCV executou em 21.97 segundos para todas as combinações de candidatos a parâmetros do modelo.


{'mean_fit_time': array([0.03219013, 0.03281236, 0.03340669, 0.03605905, 0.0361197 ,
        0.03475966, 0.0456109 , 0.04400344, 0.04438744, 0.04510412,
        0.04066811, 0.03492022, 0.05199213, 0.0469058 , 0.04411173,
        0.07694926, 0.07143846, 0.06341124, 0.03336129, 0.03229346,
        0.03251395, 0.03881707, 0.03728557, 0.03809519, 0.05396891,
        0.05288301, 0.05108442, 0.0548306 , 0.04391389, 0.03785505,
        0.06999435, 0.06001744, 0.04900951, 0.10993705, 0.09826531,
        0.08172512, 0.0256103 , 0.02495246, 0.02447295, 0.02952204,
        0.02945981, 0.03053846, 0.04336343, 0.04318523, 0.04287353,
        0.04556098, 0.03428764, 0.02843981, 0.05736828, 0.04896216,
        0.04218249, 0.09425106, 0.0891613 , 0.07784743, 0.02643008,
        0.02672906, 0.02651935, 0.03305154, 0.03362789, 0.03348136,
        0.05170941, 0.05143809, 0.05232763, 0.05892286, 0.04132347,
        0.03563447, 0.07567329, 0.06581812, 0.05444789, 0.13571582,
        0.12716832, 0.10382385]